In [1]:
!pip install --upgrade pip
!pip install -r asr/requirements.txt
!pip install --upgrade datasets evaluate jiwer gradio torchaudio==2.3.0

In [2]:
from pathlib import Path
from dotenv import load_dotenv
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import pandas as pd
from datasets import Dataset
import torchaudio
import os
import json

load_dotenv()
TEAM_NAME = os.getenv("TEAM_NAME")
TEAM_TRACK = os.getenv("TEAM_TRACK")

In [3]:
MODEL_NAME = "openai/whisper-small.en"

model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)

model.generation_config.language = "en"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
on_gcp = None not in [TEAM_NAME, TEAM_TRACK]

if on_gcp:
    # For running on GCP
    input_dir = Path(f"/home/jupyter/{TEAM_TRACK}")
    results_dir = Path(f"/home/jupyter/{TEAM_NAME}")
else:
    # For running locally
    input_dir = Path("advanced")
    results_dir = Path("results")

# Prepare training data
data = []
with open(input_dir / "asr.jsonl", "r") as f:
    for line in f:
        if line.strip() == "":
            continue
        instance = json.loads(line.strip())
        audio_path = input_dir / "audio" / instance["audio"]
        data.append((instance["key"], audio_path.as_posix(), instance["transcript"]))

df = pd.DataFrame(data, columns=["id", "audio_path", "transcript"])
dataset = Dataset.from_pandas(df)
print(dataset)

def prepare_data(data):
    wave, sr = torchaudio.load(data["audio_path"])
    # assert sr == processor.feature_extractor.sampling_rate, f"Invalid sample rate: {sr}"

    # Extract audio features
    input_features = processor(wave.squeeze().numpy(), sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt").input_features

    # Process the transcript
    labels = processor.tokenizer(data["transcript"], return_tensors="pt").input_ids

    return {"input_features": input_features.squeeze(), "labels": labels.squeeze()}

dataset = dataset.map(prepare_data, remove_columns=["id", "audio_path", "transcript"])
print(dataset)

# Split into training and testing
dataset_dict = dataset.train_test_split(test_size=0.1)
print(dataset_dict)

Dataset({
    features: ['id', 'audio_path', 'transcript'],
    num_rows: 3500
})


Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 3500
})
DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3150
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 350
    })
})


In [5]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [6]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [7]:
from transformers import Seq2SeqTrainingArguments

MODEL_NAME = "whisper-small.en-til-24"
OUTPUT_DIR = Path("./asr/models") / MODEL_NAME

training_args = Seq2SeqTrainingArguments(
    output_dir=f"{OUTPUT_DIR}",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    max_steps=100,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [8]:
trainer.train()
trainer.save_model(OUTPUT_DIR)
print('Done')

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,0.047300,0.045732,3.346124


Done
